In [ ]:
# default_exp softwares.bam.stringtie

# stringtie

In [ ]:
# export

from pybiotools.base import Base,modify_cmd

In [ ]:
# export

class Stringtie(Base):
    def __init__(self, software, fd):
        super(Stringtie, self).__init__(software)
        self._default = fd

    def cmd_version(self):
        '''
        :return:
        '''
        return 'echo {repr} ;{software} --version'.format(
            repr=self.__repr__(),
            software=self._software
        )

    @modify_cmd
    def cmd_assemble_transcript(self, bams, outgtf, abundance,annogtf):
        '''
        :param bams:
        :param outgtf:
        :param abundance:
        :param annogtf:
        :return:
        '''
        return r'''
{stringtie} {assemble_transcript} \
            -o {outgtf} \
            -G {annogtf} \
            -A {abundance} \
            {bams}        
        '''.format(
            stringtie=self._software,
            bams=bams if isinstance(bams, str) else ' '.join(bams),
            assemble_transcript=self._default['assemble_transcript'],
            outgtf=outgtf,
            annogtf=annogtf,
            abundance=abundance

        )

    @modify_cmd
    def cmd_merge_gtf(self, gtfs, output, tag=None):
        '''
        :param gtfs:
        :param output:
        :param tag:
        :param nt:
        :return:
        '''
        if tag is None:
            tag = ' -l MERGE'
        else:
            tag = ' -l ' + tag
        return r'''
{stringtie} {merge_paras} {tag} \
    -o {output} \
    {gtfs}     
    '''.format(
            stringtie=self._software,
            gtfs=gtfs if isinstance(gtfs, str) else ' '.join(gtfs),
            merge_paras=self._default.merge,
            nt=self._default.nt if None is nt else nt,
            output=output,
            tag=tag

        )

    def __repr__(self):
        return 'stringtie:' + self._software

    def __str__(self):
        return 'Transcript assembly and quantification for RNA-Seq'


In [ ]:
import configparser

config=configparser.ConfigParser()
config.read('pybiotools/default.ini')



stringtie=Stringtie('stringtie',config['stringtie'])
stringtie

stringtie:stringtie

In [ ]:
reference='biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa'
gtf_file='biology-test-data/gtf/HS.22.gtf'

outbam_pe='./pybiotools/hisat_pe.bam'
outbam_se='./pybiotools/hisat_se.bam'
bams = [outbam_pe,outbam_se]
outgtf='./pybiotools/stringtie.gtf'
abundance='./pybiotools/stringtie.txt'

In [ ]:
stringtie.cmd_version()

'echo stringtie:stringtie ;stringtie --version'

In [ ]:
print(stringtie.cmd_assemble_transcript(bams, outgtf, abundance,gtf_file))

stringtie -p 16 -e \
            -o ./pybiotools/stringtie.gtf \
            -G biology-test-data/gtf/HS.22.gtf \
            -A ./pybiotools/stringtie.txt \
            ./pybiotools/hisat_pe.bam ./pybiotools/hisat_se.bam
